In [1]:
import pandas as pd
from torch import nn
import torch
import numpy as np
import random
import  matplotlib.pyplot as plt
from torch.autograd import Function
from torch.autograd import Variable
from datetime import datetime
import math
import csv

In [2]:
all_data = pd.read_csv("../../SongApp.csv",encoding ='utf_8',delimiter=',')

In [3]:
all_data = all_data.drop(['Unnamed: 0'],axis = 1, inplace = False)

In [4]:
mat = all_data.as_matrix()

In [5]:
new_data_l = []
for i in range(50000):
    indice = random.randrange(0,len(mat))
    ligne = mat[indice]
    new_data_l.append(ligne)
new_data = pd.DataFrame(new_data_l)

In [6]:
def normalise_labels(labels):
    new_label = []
    mini = min(labels)
    maxi = max(labels)
    for i in range(len(labels)):
        new_label.append((labels[i]-mini)/(maxi-mini))
    return new_label,maxi,mini

def normalise_data(data):
    maxi = np.array(data).max()
    mini = np.array(data).min()
    new_data = []
    for i in range(len(data)):
        ligne = []
        for j in range(len(data[i])):
            ligne.append((data[i][j]-mini)/(maxi-mini))
        new_data.append(ligne)
    return new_data

def taux_erreur(modele ,data,label,loss):
    erreur = 0
    for i in range(len(data)):
        x = torch.FloatTensor(data[i])
        x = Variable(x.view(-1,taille_vect),requires_grad=True)
        y = torch.FloatTensor(1)
        y.zero_()
        y[0] = (label[i])
        y = Variable(y,requires_grad=False)
        y_pred = modele(x)
        err  = loss(y_pred , y)
        erreur+= err.data[0]
    return erreur/(len(data)*1.0)

In [7]:
labels,maxi,mini = normalise_labels(new_data[0].as_matrix())
data = normalise_data(new_data.drop([0],axis = 1, inplace = False).as_matrix())
taille_vect = len(data[0])

In [8]:
class Linear_Model(torch.nn.Module):
    def __init__(self , X ,Y) :
        torch.nn.Module.__init__(self)
        self.modele = torch.nn.Linear(X,Y)    
        
    def forward(self , x):
        return (torch.tanh(self.modele(x)))

In [9]:
class Out_Model(torch.nn.Module):
    def __init__(self , X ,Y) :
        torch.nn.Module.__init__(self)
        self.modele = torch.nn.Linear(X,Y)
        
    def forward(self , x):
        return (torch.sigmoid(self.modele(x)))

In [10]:
def train(modele , loss,data,nb_iter,step,label):
    tab_err_train = []
    i = 0
    data_train = []
    label_train = []
    while i < nb_iter :
        indice = random.randint(0,len(data)-1)
        x = torch.FloatTensor(data[indice])
        x = Variable(x.view(-1,taille_vect))
        y = torch.FloatTensor(final_out)
        y[0] = label[indice]
        y = Variable(y)
        y_pred = modele(x)
        erreur  = loss(y_pred , y)
        modele.zero_grad()
        #Since the backward() function accumulates gradients, and you don’t want to mix up gradients between minibatches, you have to zero them out at the start of a new minibatch. This is exactly like how a general (additive) accumulator variable is initialized to 0 in code.
        erreur.backward()
        for param in modele.parameters():
            param.data -= step * param.grad.data
        i+=1
        if(i % 2 ==0):
            err = taux_erreur(modele ,data,label,loss)
            print ('    Erreur en train : ',err)
            print(' iter : ',i)
    return tab_err_train

In [11]:
final_out = 1
output = 1500 #nb neuronne couche caché
models = {}
loss = torch.nn.MSELoss()
modele = torch.nn.Sequential(Linear_Model(taille_vect,output),Linear_Model(output,output),Out_Model(output,final_out))

In [14]:
step = 0.08 #pas de gradient
nb_iter =500
tab_err_train = train(modele,loss,data,nb_iter,step,labels)

    Erreur en train :  0.017261366488444466
 iter :  2
    Erreur en train :  0.01758071853541987
 iter :  4
    Erreur en train :  0.01755181617872147
 iter :  6
    Erreur en train :  0.01777100967482351
 iter :  8
    Erreur en train :  0.017143109002571034
 iter :  10
    Erreur en train :  0.01672743105203394
 iter :  12
    Erreur en train :  0.017237455623325248
 iter :  14
    Erreur en train :  0.01758328800505112
 iter :  16
    Erreur en train :  0.015927999652073463
 iter :  18
    Erreur en train :  0.015800891007652394
 iter :  20
    Erreur en train :  0.015933918431953496
 iter :  22
    Erreur en train :  0.015753334743350857
 iter :  24
    Erreur en train :  0.015802813870928037
 iter :  26
    Erreur en train :  0.015977587105236158
 iter :  28
    Erreur en train :  0.015945550654402348
 iter :  30
    Erreur en train :  0.016161785894192034
 iter :  32
    Erreur en train :  0.016205130237561587
 iter :  34
    Erreur en train :  0.015873603941426555
 iter :  36
 

    Erreur en train :  0.016816945376493444
 iter :  294
    Erreur en train :  0.016863756689059015
 iter :  296
    Erreur en train :  0.017294430558152857
 iter :  298
    Erreur en train :  0.0171367999972596
 iter :  300
    Erreur en train :  0.017268279970728603
 iter :  302
    Erreur en train :  0.01716469419615136
 iter :  304
    Erreur en train :  0.017512268241480725
 iter :  306
    Erreur en train :  0.017130789076604397
 iter :  308
    Erreur en train :  0.017484733578967102
 iter :  310
    Erreur en train :  0.016044680712246774
 iter :  312
    Erreur en train :  0.015799118446913465
 iter :  314
    Erreur en train :  0.015828437794788687
 iter :  316
    Erreur en train :  0.01581955284317695
 iter :  318
    Erreur en train :  0.01579647523236857
 iter :  320
    Erreur en train :  0.01571876262083209
 iter :  322
    Erreur en train :  0.01576158809253214
 iter :  324
    Erreur en train :  0.015741552942621292
 iter :  326
    Erreur en train :  0.0157177871225

In [15]:
def inference(data,maxi,mini,label):
    x = torch.FloatTensor(data)
    x = Variable(x.view(-1,taille_vect))
    y_pred = modele(x)
    y_denormalise = round((y_pred*(maxi-mini)+mini).data[0][0])
    label_denormalise = (label*(maxi-mini)+mini)
    print ("Valeure predite : ",(y_denormalise)," Valeure attendue : ",label_denormalise)

In [18]:
for i in range(10000):
    inference(data[i],maxi,mini,labels[i])

Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  1991.0
Valeure predite :  1996  Valeure attendue :  1997.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  1968.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  1989.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  1978.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  2010.0
Valeure predite :  1996  Valeure attendue :  1995.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  1999.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  2000.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2000.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  1972.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  1975.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  1997.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2009.0
Valeure predite :  1996  Valeure attendue :  1999.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  1997.0
Valeure predite :  1996  Valeure attendue :  1989.0
Valeure predite :  1996  Valeure attendue :  2010.0
Valeure predite :  1996  Valeure attendue :  1992.0
Valeure predite :  1996  Valeure attendue :  2009.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  1995.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  1990.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  1999.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  1964.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  1999.0
Valeure predite :  1996  Valeure attendue :  1989.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  1970.0
Valeure predite :  1996  Valeure attendue :  1996.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  1986.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  1996.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  1992.0
Valeure predite :  1996  Valeure attendue :  2000.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure pred

...

Valeure predite :  1996  Valeure attendue :  1991.0
Valeure predite :  1996  Valeure attendue :  1999.0
Valeure predite :  1996  Valeure attendue :  1973.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  1992.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2010.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  1985.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  1997.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2009.0
Valeure predite :  1996  Valeure attendue :  2000.0
Valeure predite :  1996  Valeure attendue :  1993.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  1960.0
Valeure predite :  1996  Valeure attendue :  1979.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  1990.0
Valeure predite :  1996  Valeure attendue :  1997.0
Valeure predite :  1996  Valeure attendue :  1976.0
Valeure predite :  1996  Valeure attendue :  1993.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  1991.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2009.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  1974.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  2008.0
Valeure predite :  1996  Valeure attendue :  1984.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  1987.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  2000.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  1989.0
Valeure predite :  1996  Valeure attendue :  2000.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  2009.0
Valeure predite :  1996  Valeure attendue :  1992.0
Valeure predite :  1996  Valeure attendue :  2007.0
Valeure predite :  1996  Valeure attendue :  1989.0
Valeure predite :  1996  Valeure attendue :  2005.0
Valeure predite :  1996  Valeure attendue :  2008.0
Valeure pred

Valeure predite :  1996  Valeure attendue :  1990.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  2010.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  1991.0
Valeure predite :  1996  Valeure attendue :  1992.0
Valeure predite :  1996  Valeure attendue :  1994.0
Valeure predite :  1996  Valeure attendue :  2003.0
Valeure predite :  1996  Valeure attendue :  2009.0
Valeure predite :  1996  Valeure attendue :  2006.0
Valeure predite :  1996  Valeure attendue :  1987.0
Valeure predite :  1996  Valeure attendue :  2004.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  1998.0
Valeure predite :  1996  Valeure attendue :  1995.0
Valeure predite :  1996  Valeure attendue :  2001.0
Valeure predite :  1996  Valeure attendue :  2002.0
Valeure predite :  1996  Valeure attendue :  1976.0
Valeure pred